In [1]:
from _18color_faiss import BaseMain

cfg = {
        "mode" : "train", #train, #infer
        
        "model_name" : "seresnext50_32x4d", #"tf_efficientnetv2_m.in21k", #"swinv2_base_window12to16_192to256_22kft1k",
        # "tf_efficientnet_b4", #"resnetv2_101x1_bit", #"resnetv2_152x2_bit", #"resnetv2_50x1_bit",
        #"tf_efficientnetv2_s.in21k",#"eva_large_patch14_196.in22k_ft_in1k",, #"wide_resnet101_2", #"seresnext50_32x4d"
        #"beit_base_patch16_224.in22k_ft_in22k", #"convnextv2_base.fcmae_ft_in1k", #"seresnet101", #"seresnext101_64x4d"
        #"nf_resnet50",#"nfnet_f1"
        "num_classes" : 18,
        
        "learning_rate" : 5e-4,
        "focal_alpha" : 2,
        "focal_gamma" : 2,
        "resize" : 224,
        
        "data_train_path" : "./Dataset/Train",
        "data_train_csv_path" : "./Dataset/aug_info_etri20_color_train.csv",
        "data_valid_path" : "./Dataset/Validation",
        "data_valid_csv_path" : "./Dataset/info_etri20_color_validation.csv",
        
        "data_infer_path" : "/aif/Dataset/Test/",
        "data_infer_csv_path" : "/aif/Dataset/info_etri20_color_test.csv",
        # "data_infer_path" : "./sub-task2/Dataset/Test_sample",
        # "data_infer_csv_path" : "./sub-task2/Dataset/info_etri20_color_test_sample.csv",
        
        "epochs" : 80,
        "batch_size" : 32,
        "num_worker" : 4,
        "early_stop_patient" : 30,
        
        "reuse" : True, #True, #False
        "weight_path" : "./ckpt/seresnext50_32x4d/SpatialAug_SmallSize_GridTTA/77E-val0.6163044630766803-seresnext50_32x4d.pth",
        
        "save_path" : "./ckpt/seresnext50_32x4d/OfflineAug",
        "output_path" : "./output/seresnext50_32x4d/OfflineAug",
        "log_path" : "./logging/OfflineAug",
        "device" : "cuda",
        
        "binary_mode" : False,
        
        "loss_weight" : [0.8622, 1.2872, 1.0935, 0.9228, 0.9122, 1.0983, 1.1635, 1.0886,
                        1.015 , 1.1258, 0.9061, 0.9446, 0.9728, 0.8595, 0.9743, 0.9163,
                        0.9292, 0.9281],
        
        
        "seed": 2455,
        "note" : ["offline aug for distance smapling data",
                "Cutmix 사용", "CE Loss 사용", "Adam Optim 사용"]
    } 

cfg["shuffle"] = True

In [2]:
base_main = BaseMain(**cfg)

In [3]:
import cv2
import os
import albumentations as A
from tqdm import tqdm
from glob import glob
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2

resize = 224
transform = A.Compose([
    A.Resize(resize, resize),
    A.Normalize(),
    # A.Normalize(mean=(0.548172032,0.467046563,0.434142448),
    #             std=(0.12784231,0.122905336,0.119736256)),
    ToTensorV2()
])



In [15]:
cnt = 1
embs = []
labels = []
for img, label in base_main.valid_loader :
    img = img.to(cfg["device"])
    emb = base_main.model(img)
    
    print(emb.shape)
    print(label)
    
    embs.append(emb.detach().cpu().numpy())
    labels.append(label.detach().cpu().numpy())
    if cnt == 10 :
        break
    cnt += 1

torch.Size([32, 18])
tensor([ 3,  8,  9,  3, 16, 16, 16,  8, 16,  3,  7,  7,  3, 11, 12,  1, 10,  1,
        16, 17,  6,  8, 15, 13,  0,  5,  2,  8,  9,  7, 11,  0])
torch.Size([32, 18])
tensor([ 9, 10,  5,  0,  0, 16,  1, 11,  5,  2,  0, 13,  1, 10, 10, 13,  0,  8,
         3,  0, 12,  6, 10, 14, 11, 15,  6,  9, 15,  2, 17,  7])
torch.Size([32, 18])
tensor([ 0, 17,  1, 17,  9, 12,  0,  0, 12,  6, 14,  5,  1, 12,  9,  8, 17,  0,
        11, 16,  9,  0, 13,  8, 11,  0,  4, 17, 17,  7,  0,  4])
torch.Size([32, 18])
tensor([ 9,  0,  4, 12,  0, 13,  0, 11, 11, 11, 11,  8, 13, 14,  1, 13, 11,  4,
         2, 10,  3,  9, 15, 17,  1, 14,  1, 16, 12, 15,  5, 15])
torch.Size([32, 18])
tensor([15, 17,  6, 10,  0, 12,  5,  4, 12, 11, 11, 17,  1, 10, 10,  1,  9,  5,
         1,  7,  7,  6, 10,  6,  7,  7,  8, 14,  0, 16, 11,  1])
torch.Size([32, 18])
tensor([16,  6,  1,  4, 14, 12, 11,  7, 13,  3, 12,  3, 11,  9,  5,  0,  7,  6,
         8, 12, 15, 16,  5, 12,  3,  5,  3,  7,  5,  0,  6,  7])
torc

In [16]:
import numpy as np
_embs = np.concatenate([*embs], axis=0)
_labels = np.concatenate([*labels], axis=0)
print(_embs.shape)
print(_labels.shape)

(320, 18)
(320,)


In [17]:
import umap

u = umap.UMAP()
embedding = u.fit_transform(_embs, _labels)
embedding.shape

(320, 2)

In [18]:
import faiss
# _emb = emb.detach().cpu().numpy()
v = embedding[32:]
q = embedding[:32]

ncentroids = 18
niter = 20
verbose = True
d = v.shape[1]
kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose)
kmeans.train(v)

output = kmeans.index.search(q, 1)


In [19]:
output[0].reshape(-1)

array([7.6818657e-01, 4.2748642e-01, 1.0984039e-01, 6.2202835e-01,
       1.5490580e+00, 2.3709488e-01, 8.7512970e-02, 1.8310547e-04,
       1.7451048e+00, 1.0868359e+00, 5.7999802e-01, 4.3184662e-01,
       9.3663597e-01, 3.6287308e-02, 2.1261597e-01, 6.7300415e-01,
       9.8933029e-01, 4.4744873e-01, 1.4165373e+00, 2.5086975e-01,
       4.4952774e-01, 4.1925240e-01, 1.0943222e-01, 6.0192966e-01,
       9.7041321e-01, 2.4757385e-03, 8.3040619e-01, 2.3669052e-01,
       3.4387970e-01, 1.3944473e+00, 1.0197449e-01, 5.7863617e-01],
      dtype=float32)

In [20]:
output[1].reshape(-1)

array([ 3,  2, 15, 10, 13, 17,  8,  4, 13, 10, 10, 10, 10, 11,  2,  5, 11,
        5, 13,  9,  2, 10,  0,  3, 11, 12, 15,  2, 15,  6, 11, 12],
      dtype=int64)

In [11]:
tensor([ 3,  8,  9,  3, 16, 16, 16,  8, 16,  3,  7,  7,  3, 11, 12,  1, 10,  1,
        16, 17,  6,  8, 15, 13,  0,  5,  2,  8,  9,  7, 11,  0])

tensor([10,  7,  8, 14, 13, 17, 16, 15, 11,  7,  3, 13, 13,  6,  4, 15,  5,  6,
        13,  1, 12,  8,  2, 15,  1,  3, 16, 14, 16, 14, 11, 15])

In [29]:
print(kmeans.k)
print(kmeans.d)
print(kmeans.centroids)
print(dir(kmeans.index))

18
2
[[ 2.77829289e+00 -1.78601861e+00]
 [ 2.04424691e+00  4.55984211e+00]
 [-3.43056393e+00  4.04125309e+00]
 [-7.73954717e-03  1.87284362e+00]
 [ 1.80512943e+01  2.44813770e-01]
 [-9.01733589e+00  1.31401405e+01]
 [ 4.42787933e+00  2.32777262e+00]
 [-2.74525452e+00  8.20581317e-01]
 [-3.99737048e+00  2.77527118e+00]
 [-4.77781248e+00  3.43265629e+00]
 [ 3.69156361e+00 -5.97504497e-01]
 [-4.99571413e-02  3.98870254e+00]
 [ 1.34061837e+00  4.70820141e+00]
 [ 1.97944522e+00  5.31260490e-01]
 [-4.93685102e+00  1.23555765e+01]
 [ 3.98814249e+00  4.97100973e+00]
 [-1.89098418e+00  5.68493414e+00]
 [-1.23347187e+00  3.11454535e+00]]
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '_

: 